<div class="row">
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# [SIMULATIONS] FAST-OAD with BWB integrated architecture
#### Sandra MUÑOZ 
#### Justo RODRÍGUEZ
#### Miguel VALADAS

Latest update: 26 March 2023

--------------------------

This notebook evaluates the problem for a list of varying inputs. This script was created for the sensitivity analysis. 

## 1. Import libraries

In [1]:
# Check if all packages are installed. If not, the program will automatically install the missing ones. 

import importlib.util
import sys
import subprocess
# Packages list
package_names = ['numpy', 'os.path','openmdao.api','logging','shutil','fastoad.api','math','matplotlib.pyplot','plotly.graph_objects','array','time','sys','pandas','qgrid','yaml']
num_packages = len(package_names)

# Function to install package
def install(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    
# Test libraries: 
for i in range(num_packages):
    spec = importlib.util.find_spec(package_names[i])
    if spec is None:
        print("-----------ERROR-----------: Package '" + package_names[i] +"'  is not installed.")
        # Install package: 
        install(package_names[i])
        print("-----------UPDATE-----------: Package '" + package_names[i] +"'  is now correctly installed.")
        
    else:
        print("Package '" + package_names[i] + "' is correctly installed.")
        
# Import packages once all of them are correctly installed: 
import openmdao.api as om
from fastoad.module_management.constants import ModelDomain
from fastoad.module_management.service_registry import RegisterOpenMDAOSystem, RegisterSubmodel
import logging
import shutil
import fastoad.api as oad
import os.path as pth
import os
import yaml
from ruamel.yaml import YAML
import matplotlib.pyplot as plt
from IPython.display import Javascript, display, Image, HTML
import numpy as np
import math
import plotly.graph_objects as go
import array
import time
import sys
import pandas as pd
import qgrid


Package 'numpy' is correctly installed.
Package 'os.path' is correctly installed.
Package 'openmdao.api' is correctly installed.
Package 'logging' is correctly installed.
Package 'shutil' is correctly installed.
Package 'fastoad.api' is correctly installed.
Package 'math' is correctly installed.
Package 'matplotlib.pyplot' is correctly installed.
Package 'plotly.graph_objects' is correctly installed.
Package 'array' is correctly installed.
Package 'time' is correctly installed.
Package 'sys' is correctly installed.
Package 'pandas' is correctly installed.
Package 'qgrid' is correctly installed.
Package 'yaml' is correctly installed.


## 2. Directory set up 

In [2]:
# Print the current working directory
directory = "{0}".format(os.getcwd())
print('Directory: ', directory)

# Assign data and work folder paths: 
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workspace"

# Print data and work folder paths: 
print('- Data folder path is: ',directory +'\data')
print('- Work folder path is: ',directory +'\workspace')

Directory:  C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204
- Data folder path is:  C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\data
- Work folder path is:  C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\workspace


## 3. Initialization

In [3]:
# For having log messages on screen
# logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

##################################################################################

CONFIGURATION_FILE_BWB = pth.join(WORK_FOLDER_PATH, "BWB_configuration.yml")
SOURCE_FILE_BWB = pth.join(DATA_FOLDER_PATH, "BWB_inputs.xml")
# oad.generate_configuration_file(CONFIGURATION_FILE_tubeWing, overwrite=True)

print('Problem initialized correctly')

Problem initialized correctly


## 4. Generate inputs

In [4]:
# Change inputs and outputs file names in the configuration file:
yaml = YAML()
yaml.preserve_quotes = True
with open(CONFIGURATION_FILE_BWB) as f:
    test = yaml.load(f)
test['input_file'] = './problem_inputs_BWB.xml'
test['output_file'] = './problem_outputs_BWB.xml'
with open(CONFIGURATION_FILE_BWB, 'wb') as f:
    yaml.dump(test, f)

# Generate inputs (reference case): 
inputsBWB = oad.generate_inputs(CONFIGURATION_FILE_BWB, SOURCE_FILE_BWB, overwrite=True)
print(' - BWB inputs directory is:\n', inputsBWB)

# Visualize list of inputs:
oad.variable_viewer(inputsBWB)

 - BWB inputs directory is:
 C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\workspace\problem_inputs_BWB.xml


## 5. Evaluate problem

#### PROBLEM PARAMETERS 
In this part the user must choose the problem parameters: 
 - Number of simulations to be performed.
 - Vector of inputs of interest.
 
 As an example, the wing sweep angle variation is given.

In [5]:
# Set up simulation parameters: 
print('-------------------------------------------------------------')
print('SIMULATION INFORMATION: ')
print('-------------------------------------------------------------')
nsim = 21 # Number of simulations to be performed
simDir = directory +'\workspace\BWBsimulations' # The inputs and outputs of the simulation will be stored in this folder
CONFIGURATION_FILE_sims = pth.join(simDir, "BWB_configuration.yml")
input_data = oad.DataFile(inputsBWB)

# WING SWEEP ANGLE EXAMPLE: 
sweep0 = 15 # Initial value of the vector
sweep_increment = 1 # Step
sweepfinal = sweep0+sweep_increment*(nsim-1) # Final value of the vector
sweep_vec = np.linspace(sweep0,sweepfinal,nsim) # Vector of inputs
# sweep_orig = sweep_vec[10] # Default wing sweep angle (reference simulation)

print('- A total of', nsim,'simulations will be performed.')
print('- BWB simulations directory is: ', simDir)
print('- The sweep vector is: ', sweep_vec)


-------------------------------------------------------------
SIMULATION INFORMATION: 
-------------------------------------------------------------
- A total of 21 simulations will be performed.
- BWB simulations directory is:  C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\workspace\BWBsimulations
- The sweep vector is:  [15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.
 33. 34. 35.]


#### RUN SIMULATION - SWEEP ANGLE EXAMPLE

In [6]:
#  RUN SIMULATION
for i in range(0, nsim):
    print('-------------------------------------------------------------')
    print('START OF SIMULATION ', i+1, ', Sweep =', sweep_vec[i], 'deg')
    print('-------------------------------------------------------------')
    
#     Change inputs and outputs file names in the configuration file: 
    yaml = YAML()
    yaml.preserve_quotes = True
    with open(CONFIGURATION_FILE_sims) as f:
        test = yaml.load(f)
    test['input_file'] = './problem_inputs_BWB_sweep%d.xml' %(sweep_vec[i]) 
    test['output_file'] = './problem_outputs_BWB_sweep%d.xml' %(sweep_vec[i]) 
    with open(CONFIGURATION_FILE_sims, 'wb') as f:
        yaml.dump(test, f)
    
#   Generate new inputs with the name given above:
    CONFIGURATION_FILE_BWB = pth.join(simDir, "BWB_configuration.yml")
    SOURCE_FILE_BWB = pth.join(DATA_FOLDER_PATH, "BWB_inputs.xml") 
    oad.generate_inputs(CONFIGURATION_FILE_sims, SOURCE_FILE_BWB, overwrite=True)
    
#   Change the input of interest (in this case the wing sweep angle) and replace it by the corresponding vector value:              
    input_data = oad.DataFile(pth.join(simDir, "problem_inputs_BWB_sweep%d.xml" %(sweep_vec[i])))
    input_data["data:geometry:wing:sweep_25"].value[0] = sweep_vec[i]
    input_data.save() # Save the inputs
    
#   Evaluate the problem with the new inputs.xml
    eval_problem = oad.evaluate_problem(CONFIGURATION_FILE_sims, overwrite=True)
    
#   Access the outputs list:
    output_data = oad.DataFile(pth.join(simDir, "problem_outputs_BWB_sweep%d.xml"%(sweep_vec[i])))
#   Save the outputs file:
    output_data.save()
    
    print('-------------------------------------------------------------')
    print('END OF SIMULATION %d' % (i+1))
    print('-------------------------------------------------------------')


print('-------------------------------------------------------------')
print('ALL SIMULATIONS COMPLETED')
print('-------------------------------------------------------------')

-------------------------------------------------------------
START OF SIMULATION  1 , Sweep = 15.0 deg
-------------------------------------------------------------


C:\Users\migue\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\anaconda3\lib\site-packages\fastoad_cs25\models\propulsion\fuel_propulsion\rubber_engine\openmdao.py
Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\modules\propulsion\fuel_propulsion\rubber_engine\openmdao.py


NL: NLBGS Converged in 17 iterations
-------------------------------------------------------------
END OF SIMULATION 1
-------------------------------------------------------------
-------------------------------------------------------------
START OF SIMULATION  2 , Sweep = 16.0 deg
-------------------------------------------------------------


Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\anaconda3\lib\site-packages\fastoad_cs25\models\propulsion\fuel_propulsion\rubber_engine\openmdao.py
Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\modules\propulsion\fuel_propulsion\rubber_engine\openmdao.py


NL: NLBGS Converged in 17 iterations
-------------------------------------------------------------
END OF SIMULATION 2
-------------------------------------------------------------
-------------------------------------------------------------
START OF SIMULATION  3 , Sweep = 17.0 deg
-------------------------------------------------------------


...

NL: NLBGS Converged in 14 iterations
-------------------------------------------------------------
END OF SIMULATION 19
-------------------------------------------------------------
-------------------------------------------------------------
START OF SIMULATION  20 , Sweep = 34.0 deg
-------------------------------------------------------------


Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\anaconda3\lib\site-packages\fastoad_cs25\models\propulsion\fuel_propulsion\rubber_engine\openmdao.py
Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\modules\propulsion\fuel_propulsion\rubber_engine\openmdao.py


NL: NLBGS Converged in 14 iterations
-------------------------------------------------------------
END OF SIMULATION 20
-------------------------------------------------------------
-------------------------------------------------------------
START OF SIMULATION  21 , Sweep = 35.0 deg
-------------------------------------------------------------


Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\anaconda3\lib\site-packages\fastoad_cs25\models\propulsion\fuel_propulsion\rubber_engine\openmdao.py
Variable "data:propulsion:rubber_engine:turbine_inlet_temperature" out of bound: value [1633.] degK is over upper limit ( 1600.0 ) in file C:\Users\migue\Documents\MV\ISAE Supaero\Research Project\Meeting MAE1 1204\modules\propulsion\fuel_propulsion\rubber_engine\openmdao.py


NL: NLBGS Converged in 15 iterations
-------------------------------------------------------------
END OF SIMULATION 21
-------------------------------------------------------------
-------------------------------------------------------------
ALL SIMULATIONS COMPLETED
-------------------------------------------------------------
